In [ ]:


!pip install -q gradio pillow numpy opencv-python

import numpy as np
from PIL import Image
import cv2
import gradio as gr
import os


def blend_images(img1, img2, alpha=0.5, size=512):
    i1 = img1.convert("RGB").resize((size,size), Image.LANCZOS)
    i2 = img2.convert("RGB").resize((size,size), Image.LANCZOS)
    return Image.blend(i1, i2, alpha)

def sharpen_image(pil_img, amount=1.5):
    img = np.array(pil_img, dtype=np.float32)
    blur = cv2.GaussianBlur(img, (0,0), sigmaX=3)
    sharp = cv2.addWeighted(img, 1+amount, blur, -amount, 0)
    return Image.fromarray(np.clip(sharp,0,255).astype(np.uint8))

def apply_mask(img, mask_path, size=512):
    """
    Apply a white-on-black mask to the image.
    White = keep, Black = transparent (background)
    """
    img = img.convert("RGBA").resize((size,size), Image.LANCZOS)
    mask = Image.open(mask_path).convert("L").resize((size,size), Image.LANCZOS)
    img.putalpha(mask)
    return img


def fuse_images_with_shape(img1, img2, attribute, alpha=0.5, sharpen_amt=1.5):
    # Map attribute to mask file
    mask_map = {
        "boy kurthys": "masks/boy_kurthys_mask.png",
        "sheets": "masks/sheets_mask.png",
        "t-shirts": "masks/tshirts_mask.png",
        "women lehenga": "masks/women_lehenga_mask.png",
        "sarees": "masks/sarees_mask.png",
        "tops": "masks/tops_mask.png",
        "night dress": "masks/nightdress_mask.png"
    }


    if attribute in mask_map and os.path.exists(mask_map[attribute]):
        img1_masked = apply_mask(img1, mask_map[attribute])
        img2_masked = apply_mask(img2, mask_map[attribute])
    else:
        img1_masked = img1
        img2_masked = img2


    fused = blend_images(img1_masked.convert("RGB"), img2_masked.convert("RGB"), alpha)
    fused = sharpen_image(fused, amount=sharpen_amt)
    return fused

def generate(image1, image2, attribute, alpha, sharpen):
    if image1 is None or image2 is None:
        return None
    im1 = Image.fromarray(image1.astype(np.uint8))
    im2 = Image.fromarray(image2.astype(np.uint8))
    return fuse_images_with_shape(im1, im2, attribute, alpha=float(alpha), sharpen_amt=float(sharpen))

gr.Interface(
    fn=generate,
    inputs=[
        gr.Image(type="numpy", label="Image 1"),
        gr.Image(type="numpy", label="Image 2"),
        gr.Dropdown(choices=["boy kurthys","sheets","t-shirts","women lehenga","sarees","tops","night dress"],
                    label="Select Attribute"),
        gr.Slider(0.0, 1.0, 0.5, step=0.05, label="Blend Alpha"),
        gr.Slider(0.0, 3.0, 1.5, step=0.1, label="Sharpen Amount")
    ],
    outputs=gr.Image(type="pil", label="Fused & Sharpened Image"),
    title="Offline Fashion Fusion with Attribute Shape",
    description="Fuses 2 images based on selected fashion attribute and applies attribute-specific shape masks. Fully offline."
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a74812b50e8c775964.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
